<strong><div dir=rtl >
<font size=4 >3) آماده سازی داده ها  <font><strong>

<div style="text-align: justify; direction: rtl; font-family: 'B Nazanin'; font-size: 15px;">
این بخش به ادغام داده‌های تاریخی قیمت سهام فولاد و داده‌های تحلیل احساسات اخبار با هدف هم‌تاریخ‌سازی و هماهنگ‌سازی دو مجموعه داده برای مراحل بعدی پیش‌بینی مرتبط با قیمت سهام است. در این بخش، ابتدا داده‌ها و تاریخ‌های آنها تنظیم و سپس با هم ترکیب می‌شوند.

In [ ]:
import pandas as pd
historical_data=pd.read_csv('/content/drive/MyDrive/Foolad_stock.csv')

historical_data.drop('date', axis=1, inplace=True)

historical_data

,open,high,low,adjClose,value,volume,count,yesterday,close,jdate
0,2268.0,2303.0,2239.0,2266.0,710444880000,60503628,9077,2268.0,2249.0,1400-08-23
1,2249.0,2251.0,2154.0,2178.0,809287914550,71773060,10567,2266.0,2172.0,1400-08-24
2,2135.0,2199.0,2135.0,2166.0,516800766400,46062879,7117,2178.0,2153.0,1400-08-25
3,2153.0,2189.0,2133.0,2154.0,597674516320,53548043,8786,2166.0,2162.0,1400-08-26
4,2127.0,2160.0,2048.0,2064.0,1438134421010,134548898,17586,2154.0,2064.0,1400-08-29
...,...,...,...,...,...,...,...,...,...,...
679,4173.0,4275.0,4104.0,4178.0,772436478828,184882653,5385,4204.0,4224.0,1403-08-16
680,4220.0,4303.0,4220.0,4300.0,659303749349,153332821,2262,4178.0,4303.0,1403-08-19
681,4525.0,4528.0,4427.0,4476.0,2418492237969,540351588,12560,4300.0,4449.0,1403-08-21
682,4481.0,4529.0,4423.0,4471.0,1263177164076,282506106,8087,4476.0,4446.0,1403-08-22


In [ ]:
sentiment_data=pd.read_csv('/content/drive/MyDrive/sentiment_results_with_probs.csv')

In [ ]:
pip install persiantools

<div style="text-align: justify; direction: rtl; font-family: 'B Nazanin'; font-size: 15px;">
در این مرحله، تاریخ‌های موجود در داده‌های اخبار که ممکن است مربوط به روزهای غیرکاری باشند (مانند تعطیلات یا آخر هفته‌ها)، با استفاده از داده‌های تاریخ‌های کاری بورس (تاریخ‌های قیمت سهام) تنظیم می‌شوند. اگر تاریخ خبری به روز کاری بورس نرسد، تاریخ به روز کاری قبلی منتقل می‌شود.

In [ ]:
from persiantools.jdatetime import JalaliDate
from datetime import timedelta

# Convert strings to JalaliDate objects
all_dates = [JalaliDate(*map(int, date.split('/'))) for date in sentiment_data['Date']]
working_dates = [JalaliDate(*map(int, date.split('-'))) for date in historical_data['jdate']]

def adjust_non_working_dates(all_dates, working_dates):
    # Create a set of working dates for faster lookup
    working_dates_set = set(working_dates)
    adjusted_dates = []

    for date in all_dates:
        # Safeguard to prevent infinite loop
        while date not in working_dates_set:
            # Stop if we go out of range (e.g., before all working dates)
            if date < min(working_dates):
                break
            date -= timedelta(days=1)
        adjusted_dates.append(date)

    return adjusted_dates

# Adjust the dates
adjusted_dates = adjust_non_working_dates(all_dates, working_dates)

# Convert JalaliDate objects back to strings
adjusted_dates = [date.strftime("%Y-%m-%d") for date in adjusted_dates]


<div style="text-align: justify; direction: rtl; font-family: 'B Nazanin'; font-size: 15px;">
پس از تنظیم تاریخ‌ها، داده‌های تحلیل احساسات به داده‌های قیمت سهام بر اساس تاریخ مشترک ادغام می‌شوند.
در اینجا، تاریخ‌های اخبار به روزهایی که در داده‌های قیمت سهام وجود دارند، هم‌تاریخ می‌شوند. سپس، ستون‌های غیرضروری از داده‌ها حذف شده و تنها ویژگی‌های مهم نگه‌داشته می‌شوند. در ادامه، تعداد اخبار برای هر روز محاسبه شده و داده‌ها بر اساس تاریخ گروه‌بندی و میانگین‌گیری می‌شوند. در نهایت، داده‌های نهایی شامل احساسات اخبار و ویژگی‌های قیمت سهام برای هر تاریخ در یک فایل CSV ذخیره می‌شود تا برای تحلیل‌های پیش‌بینی مورد استفاده قرار گیرد.

In [ ]:
adjusted_data=sentiment_data.copy()
adjusted_data['Date']=adjusted_dates
historical_data.rename(columns={'jdate': 'Date'}, inplace=True)
full_data=pd.merge(adjusted_data, historical_data, on="Date")

full_data = full_data.drop(
    ['Title','value','volume','count','yesterday','close','open','high','low','Summary','category','full_news']
    , axis=1)

full_data['number_of_news']=[1 for i in range(len(full_data))]

number_of_news=full_data[['Date','number_of_news']].groupby(['Date']).sum()

full_data=full_data.groupby(['Date']).mean()

full_data['number_of_news']=number_of_news

In [ ]:
full_data

,Negative,Positive,Neutral,adjClose,number_of_news
Date,,,,,
1400-08-23,0.432091,0.254842,0.313067,2266.0,67
1400-08-24,0.328261,0.348066,0.323674,2178.0,61
1400-08-25,0.376083,0.299892,0.324026,2166.0,60
1400-08-26,0.386278,0.325757,0.287965,2154.0,122
1400-08-29,0.378658,0.335817,0.285525,2064.0,60
...,...,...,...,...,...
1403-08-16,0.302686,0.210829,0.486485,4178.0,155
1403-08-19,0.410837,0.223466,0.365697,4300.0,112
1403-08-21,0.439202,0.248437,0.312360,4476.0,84


In [ ]:
full_data.to_csv('/content/drive/MyDrive/final_data.csv')